In [10]:
###############################################################################
#                             Carla CHATBOT                                   #
###############################################################################

# Setup
## for speech-to-text
import speech_recognition as sr

## for text-to-speech
from gtts import gTTS

## for language model
import transformers

## for data
import os
import time
import datetime

import numpy as np
import randfacts
import requests
from api import *
import playsound

#selenium imports 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options 

In [104]:
###############################################################################
#                             Carla CHATBOT                                   #
###############################################################################

# Build the AI
class ChatBot():
    def __init__(self, name):
        print("--- starting up", name, "---")
        self.name = name
    
    #Speech to Text Fun
    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            recognizer.dynamic_energy_threshold = True
            #recognizer.energy_threshold = 30552
            recognizer.adjust_for_ambient_noise(mic, duration = 1.5)
            print("listening...")
            audio = recognizer.listen(mic)
            print("....")
            #,timeout = 9    ,show_all = False 
        try:
            #recognizer.recognize_whisper()
            self.text = recognizer.recognize_google(audio)
            print("me --> ", self.text)
        except:
            print("me -->  ERROR")

    #Text to Speech
    @staticmethod
    def text_to_speech(text):
        print("ai --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        
        playsound.playsound('res.mp3', True)
        #os.system("start res.mp3")  # mac->afplay | windows->start
        #time.sleep(5)
        time.sleep(int(40*duration))
        
        os.remove("res.mp3")
         
    
    

    def wake_up(self, text):
        #.......
        return True if self.name in text.lower() else False

    @staticmethod
    def action_time():
        #.......
        return datetime.datetime.now().time().strftime('%H:%M')
    
    @staticmethod
    def wishme():
        #.......
        hour = int(datetime.datetime.now().hour)
        if hour> 0 and hour<12:
            return('morning')
        elif hour>=12 and hour<16:
            return('afternoon')
        else:
            return('evening')
    today_date = datetime.datetime.now()
        
            
    #@staticmethod
    def news(self, num_top_news):
        self.num_top_news = num_top_news
        ar=[]
        for i in range(num_top_news):
            ar.append("article "+str(i+1)+": "+json_data["articles"][i]["title"]+".")
        return ar 
    
    #wiki search
    def wiki_search(self, text):
        self.text = text
        driver = webdriver.Chrome()
        driver.get('http://www.wikipedia.org/');
        time.sleep(1)
        search = driver.find_element(By.XPATH,'//*[@id="searchInput"]')
        search.click()
        search.send_keys(text)
        search.send_keys(Keys.RETURN)
        input('Press Enter to continue If you Done')
        #Quits the driver and close every associated window.
        driver.quit()
    #vidoe Search
    def youtube_search(self, text):
        self.text = text
        driver = webdriver.Chrome()
        driver.get(url="https://www.youtube.com/results?search_query=" + text);
        time.sleep(1)
        search = driver.find_element(By.XPATH,'//*[@id="video-title"]')
        search.click()
        input('/.....Press Enter to continue If you Done')
        #Quits the driver and close every associated window.
        #driver.quit()

In [118]:
###############################################################################
#                             Carla CHATBOT                                   #
###############################################################################

# Run the AI
if __name__ == "__main__":
    #.......
    ai = ChatBot(name="Carla")
    nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    #News API Auth...
    news_key = "20978f1dac5f4cd6938d7b243f54fd15"
    api_add="http://newsapi.org/v2/top-headlines?country=us&apiKey=" + news_key
    json_data = requests.get(api_add).json()


    chat_exit = True
    while chat_exit:
        ai.speech_to_text()

        ## wake up
        #=========
        if any(i in ai.text for i in ["Carla","hey carla" ,"hey Carla what's up" , "what's up", "wake up" ,"hey carla wake up"]):
            #.......
            ai.text_to_speech("Hello I am Carla, good "+ai.wishme()+" I'm your voice assistant, " + "today is, " + ai.today_date.strftime('%d')
                  + ai.today_date.strftime('%B' + ', and its currently ' + ai.today_date.strftime('%I') )
                  + ai.today_date.strftime('%M') + (ai.today_date.strftime('%p') ) 
            )
            res = ("what can i do for you?")

        
        ## action time
        #==============
        elif any(i in ai.text for i in ["time" ,"what is the time"]):
                   res = ai.action_time()

                
        # respond politely
        #==================
        elif any(i in ai.text for i in ["thank","thanks"]):
                    res = np.random.choice(["you're welcome!","anytime!","no problem!","cool!","I'm here if you need me!","mention not"])

        
        # get some facts
        #================
        elif any(i in ai.text for i in ["fact","facts"]):
            ai.text_to_speech('sure sir,')
            x = randfacts.get_fact()
            res = ('Did you know that, '+x)
        
       # Search in News API !
       # ======================
        elif any(i in ai.text for i in ["news" ,"News" , "top today's news", "give me the top today's news", "to day news", "top news to day"]):
            #.......
            ai.text_to_speech('sure, how many news you want to hear.')
            
            ai.num_top_news = int(input('/...pls inter numders of news you want :'))
            arr = ai.news(ai.num_top_news)
            for i in range(len(arr)):
                ai.text_to_speech(arr[i])
                #break
            ai.text_to_speech("This is the top to days News")
            res = "what can i do for you too "
                                          

        #Open Browser and Search in wikipedia
        #====================================
        elif any(i in ai.text for i in ["search Wikipedia" ,"Wikipedia" , "can you search for ","search for"]):
            #.......
            ai.text_to_speech('Yeah sure ,you information related to which topic?')
            #listen again to search topic.....
            ai.speech_to_text()
            ai.text_to_speech('search for : ' +ai.text)
            #Call wiki_search Function() 
            ai.wiki_search(ai.text)
            ai.text_to_speech('hope it was useful')
            res = "What can i do for you"
            
            
        # Open browser and Search in youtube and play vidoe !
        #====================================================
        elif any(i in ai.text for i in ["play" ,"music" , "play youtube" ,"play youtube vidoe"]):
            #.......
            ai.text_to_speech("What video or a music would you like me to play for you")
            #listen again to search topic.....
            ai.speech_to_text()
            ai.text_to_speech('Playing : ' +ai.text)
            #Call youtube_search Function()
            ai.youtube_search(ai.text)
            ai.text_to_speech('hope it was funny')
            res = "What can i do for you"
        
        #Sleep
        #=====
        elif any(i in ai.text for i in ["go to sleep" ,"close" ,"exit"]):
                 res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])
                 
                 chat_exit = False
        
                   
                   
        #conversation Transformer Model....
        #==================================
        else:
            chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
            res = str(chat)
            res = res[res.find("bot >> ") + 6:].strip()

        ai.text_to_speech(res)

#=======================================
print("/.....Close up and go to sleep!")

--- starting up Carla ---
listening...
....


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


result2:
{   'alternative': [   {   'confidence': 0.9509452,
                           'transcript': 'hello how are you'}],
    'final': True}
me -->  hello how are you
ai -->  I'm good, how are you?
listening...
....


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


result2:
{   'alternative': [   {'confidence': 0.86867559, 'transcript': 'very good'},
                       {'transcript': "I'm very good"},
                       {'transcript': 'really good'}],
    'final': True}
me -->  very good
ai -->  Thanks! I'm glad you enjoyed it.
listening...
....
result2:
{   'alternative': [   {   'confidence': 0.92994326,
                           'transcript': 'tell me a random fact'},
                       {'transcript': 'tell me random fact'}],
    'final': True}
me -->  tell me a random fact
ai -->  sure sir,
ai -->  Did you know that, The town with the most stop signs per capita than any other in the US: LaConner, Washington
listening...
....
result2:
{   'alternative': [   {   'confidence': 0.92992628,
                           'transcript': 'tell me a random fact'}],
    'final': True}
me -->  tell me a random fact
ai -->  sure sir,
ai -->  Did you know that, Billy goats urinate on their heads to become more attractive.
listening...
....
result

...

ai -->  hope it was useful
ai -->  What can i do for you
listening...
....


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


result2:
{   'alternative': [   {   'confidence': 0.92300957,
                           'transcript': 'you are so beauty'},
                       {'transcript': 'you are so beautiful'}],
    'final': True}
me -->  you are so beauty
ai -->  You are so beautiful
listening...
....


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


result2:
{   'alternative': [   {   'confidence': 0.94560105,
                           'transcript': 'can I help you in a question and you '
                                         'will answer me or what'},
                       {   'transcript': 'can I help you any question and you '
                                         'will answer me or what'},
                       {   'transcript': 'can I ask you any question and you '
                                         'will answer me or what'},
                       {   'transcript': 'can I ask you in a question and you '
                                         'will answer me or what'},
                       {   'transcript': 'can I help you in the question and '
                                         'you will answer me or what'}],
    'final': True}
me -->  can I help you in a question and you will answer me or what
ai -->  Sure, I'll PM you.
listening...
....
result2:
{   'alternative': [   {   'confidence': 0.95467675,
